In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("spam_sms.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df.describe()


,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('v1').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('v2').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['v2'].value_counts()
    for x_label, grp in _df_2.groupby('v1')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('v1')
_ = plt.ylabel('v2')

In [6]:
df.isna().sum()

v1    0
v2    0
dtype: int64

In [7]:
df.duplicated().sum()

403

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df['v1'].value_counts()

v1
ham     4516
spam     653
Name: count, dtype: int64

In [10]:
df['spam']=df['v1'].apply(lambda x: 1 if x=='spam' else 0 )

In [11]:
df['spam']


0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: spam, Length: 5169, dtype: int64

In [12]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [14]:
lm =WordNetLemmatizer()

In [15]:
# cleaning the text
def pre_processing (word):
  word=re.sub("[^a-zA-Z]",' ',word) # remove punctuat
  word=word.lower().split() # split the sentance to be every word alone (tokenzing)
  word=[lm.lemmatize(word) for word in word if word not in stopwords.words('english')] # Lemmatization every word and remove stop words
  word=' '.join(word)
  return word

In [16]:
df['v2'][3]


'U dun say so early hor... U c already then say...'

In [17]:

pre_processing('U dun say so early hor... U c already then say...')


'u dun say early hor u c already say'

In [18]:
df['pre_processing']=df['v2'].apply(pre_processing)

In [19]:
df


,v1,v2,spam,pre_processing
0,ham,"Go until jurong point, crazy.. Available only ...",0,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,0,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,0,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,nah think go usf life around though
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,nd time tried contact u u pound prize claim ea...
5568,ham,Will Ì_ b going to esplanade fr home?,0,b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",0,pity mood suggestion
5570,ham,The guy did some bitching but I acted like i'd...,0,guy bitching acted like interested buying some...


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score

from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

In [21]:
x=df['pre_processing']
y=df['spam']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)

In [23]:
vc=TfidfVectorizer()
x_train_vec=vc.fit_transform(x_train)
x_test_vec=vc.transform(x_test)

In [24]:
x_train_vec.shape


(4135, 6164)

In [26]:
models=[
    ('LR',LogisticRegression()),
    ('NB',MultinomialNB()),
    ('RF',RandomForestClassifier()),
    ('Knn',KNeighborsClassifier()),
    ('SVC',SVC())
]

In [27]:
for model in models:
  print(model[0])

  model=model[1]

  model.fit(x_train_vec,y_train)

  y_pred=model.predict(x_test_vec)

  report_cls = classification_report(y_test, y_pred)

  print(report_cls)
  print('-'*30)

LR
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       903
           1       1.00      0.69      0.81       131

    accuracy                           0.96      1034
   macro avg       0.98      0.84      0.90      1034
weighted avg       0.96      0.96      0.96      1034

------------------------------
NB
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       903
           1       0.98      0.73      0.84       131

    accuracy                           0.96      1034
   macro avg       0.97      0.87      0.91      1034
weighted avg       0.96      0.96      0.96      1034

------------------------------
RF
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       903
           1       0.99      0.81      0.89       131

    accuracy                           0.97      1034
   macro avg       0.98      0.90      0.94      1034
wei

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
model=keras.Sequential([
    keras.layers.LSTM(64,input_shape=(6164,1) ,return_sequences=True,activation='tanh'),

    keras.layers.LSTM(32,activation='tanh'),

    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy']

                )
model.fit(x_train_vec.toarray(),y_train,batch_size=64,epochs=3,verbose=1,validation_data=(x_test_vec.toarray(), y_test))

Epoch 1/3
65/65 [==============================] - 29s 370ms/step - loss: 0.4438 - accuracy: 0.8730 - val_loss: 0.3801 - val_accuracy: 0.8733
Epoch 2/3
65/65 [==============================] - 24s 373ms/step - loss: 0.3806 - accuracy: 0.8738 - val_loss: 0.3814 - val_accuracy: 0.8733
Epoch 3/3
65/65 [==============================] - 21s 323ms/step - loss: 0.3801 - accuracy: 0.8738 - val_loss: 0.3812 - val_accuracy: 0.8733


In [30]:
model.evaluate(x_test_vec.toarray(),y_test)


33/33 [==============================] - 6s 166ms/step - loss: 0.3812 - accuracy: 0.8733


[0.3812063932418823, 0.8733075261116028]